<a href="https://colab.research.google.com/github/spiritofthegael/Automated-Labeling-Threat-Reports/blob/main/techniques_snorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Eva Domschot, 2022
!pip install TextFeatureSelection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install snorkel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 5.1 MB/s 
     |████████████████████████████████| 5.6 MB 27.6 MB/s 
     |████████████████████████████████| 14.8 MB 18.1 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive 
drive.mount('/content/gdrive')
#dataset of ~1400 threat reports labelled with MITRE ATT&CK labels
df = pd.read_csv('/content/gdrive/MyDrive/thesis_work/training_data_original.csv')
df.head()

Mounted at /content/gdrive


,Text,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,...,T1124,T1035,T1086,T1490,T1216,T1094,T1043,T1211,T1127,T1077
0,Talos Blog || Cisco Talos Intelligence Group -...,0,1,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
1,OilRig Actors Provide a Glimpse into Developme...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hogfish Redleaves Malware Threat Analysis I Ac...,1,1,0,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
3,Scheduled Transfer - Enterprise | MITRE ATT&CK...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Winlogon Helper DLL - Enterprise | MITRE ATT&C...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#code to behavior name conversion 
CODE_TACTICS = ["TA0006","TA0002","TA0040","TA0003","TA0004","TA0008","TA0005","TA0010","TA0007","TA0009","TA0011","TA0001"]
NAME_TACTICS = ["Credential Access","Execution","Impact","Persistence","Privilege Escalation","Lateral Movement","Defense Evasion","Exfiltration","Discovery","Collection","Command and Control","Initial Access"]
CODE_TECHNIQUES = ["T1066","T1047","T1156","T1113","T1067","T1037","T1033","T1003","T1129","T1492","T1044","T1171","T1014","T1501","T1123","T1133","T1109","T1099","T1069","T1114","T1163","T1025","T1116","T1093","T1178","T1013","T1192","T1489","T1206","T1063","T1080","T1167","T1165","T1137","T1089","T1487","T1214","T1119","T1115","T1103","T1007","T1040","T1135","T1120","T1082","T1071","T1053","T1162","T1176","T1106","T1058","T1202","T1024","T1091","T1005","T1140","T1195","T1190","T1219","T1079","T1036","T1055","T1205","T1218","T1038","T1050","T1010","T1032","T1062","T1182","T1029","T1004","T1009","T1076","T1131","T1181","T1483","T1185","T1021","T1207","T1107","T1145","T1112","T1491","T1155","T1217","T1183","T1085","T1031","T1092","T1222","T1179","T1019","T1042","T1117","T1054","T1108","T1193","T1215","T1101","T1177","T1125","T1144","T1045","T1016","T1198","T1087","T1090","T1059","T1482","T1175","T1020","T1070","T1083","T1138","T1191","T1188","T1074","T1049","T1064","T1051","T1497","T1102","T1104","T1480","T1204","T1196","T1057","T1141","T1041","T1060","T1023","T1026","T1122","T1015","T1212","T1210","T1142","T1199","T1098","T1170","T1048","T1097","T1110","T1001","T1039","T1078","T1073","T1068","T1208","T1027","T1201","T1187","T1486","T1488","T1174","T1002","T1081","T1128","T1056","T1203","T1168","T1100","T1186","T1184","T1095","T1075","T1012","T1030","T1028","T1034","T1499","T1065","T1197","T1088","T1493","T1132","T1500","T1223","T1213","T1194","T1200","T1485","T1130","T1022","T1189","T1498","T1158","T1221","T1134","T1209","T1111","T1159","T1136","T1018","T1046","T1052","T1105","T1084","T1160","T1484","T1220","T1173","T1008","T1096","T1124","T1035","T1086","T1490","T1216","T1094","T1043","T1211","T1127","T1077"]
NAME_TECHNIQUES = ["Indicator Removal from Tools","Windows Management Instrumentation",".bash_profile and .bashrc","Screen Capture","Bootkit","Logon Scripts","System Owner/User Discovery","Credential Dumping","Execution through Module Load","Stored Data Manipulation","File System Permissions Weakness","LLMNR/NBT-NS Poisoning and Relay","Rootkit","Systemd Service","Audio Capture","External Remote Services","Component Firmware","Timestomp","Permission Groups Discovery","Email Collection","Rc.common","Data from Removable Media","Code Signing","Process Hollowing","SID-History Injection","Port Monitors","Spearphishing Link","Service Stop","Sudo Caching","Security Software Discovery","Taint Shared Content","Securityd Memory","Startup Items","Office Application Startup","Disabling Security Tools","Disk Structure Wipe","Credentials in Registry","Automated Collection","Clipboard Data","AppInit DLLs","System Service Discovery","Network Sniffing","Network Share Discovery","Peripheral Device Discovery","System Information Discovery","Standard Application Layer Protocol","Scheduled Task","Login Item","Browser Extensions","Execution through API","Service Registry Permissions Weakness","Indirect Command Execution","Custom Cryptographic Protocol","Replication Through Removable Media","Data from Local System","Deobfuscate/Decode Files or Information","Supply Chain Compromise","Exploit Public-Facing Application","Remote Access Tools","Multilayer Encryption","Masquerading","Process Injection","Port Knocking","Signed Binary Proxy Execution","DLL Search Order Hijacking","New Service","Application Window Discovery","Standard Cryptographic Protocol","Hypervisor","AppCert DLLs","Scheduled Transfer","Winlogon Helper DLL","Binary Padding","Remote Desktop Protocol","Authentication Package","Extra Window Memory Injection","Domain Generation Algorithms","Man in the Browser","Remote Services","DCShadow","File Deletion","Private Keys","Modify Registry","Defacement","AppleScript","Browser Bookmark Discovery","Image File Execution Options Injection","Rundll32","Modify Existing Service","Communication Through Removable Media","File Permissions Modification","Hooking","System Firmware","Change Default File Association","Regsvr32","Indicator Blocking","Redundant Access","Spearphishing Attachment","Kernel Modules and Extensions","Security Support Provider","LSASS Driver","Video Capture","Gatekeeper Bypass","Software Packing","System Network Configuration Discovery","SIP and Trust Provider Hijacking","Account Discovery","Connection Proxy","Command-Line Interface","Domain Trust Discovery","Distributed Component Object Model","Automated Exfiltration","Indicator Removal on Host","File and Directory Discovery","Application Shimming","CMSTP","Multi-hop Proxy","Data Staged","System Network Connections Discovery","Scripting","Shared Webroot","Virtualization/Sandbox Evasion","Web Service","Multi-Stage Channels","Execution Guardrails","User Execution","Control Panel Items","Process Discovery","Input Prompt","Exfiltration Over Command and Control Channel","Registry Run Keys / Startup Folder","Shortcut Modification","Multiband Communication","Component Object Model Hijacking","Accessibility Features","Exploitation for Credential Access","Exploitation of Remote Services","Keychain","Trusted Relationship","Account Manipulation","Mshta","Exfiltration Over Alternative Protocol","Pass the Ticket","Brute Force","Data Obfuscation","Data from Network Shared Drive","Valid Accounts","DLL Side-Loading","Exploitation for Privilege Escalation","Kerberoasting","Obfuscated Files or Information","Password Policy Discovery","Forced Authentication","Data Encrypted for Impact","Disk Content Wipe","Password Filter DLL","Data Compressed","Credentials in Files","Netsh Helper DLL","Input Capture","Exploitation for Client Execution","Local Job Scheduling","Web Shell","Process Doppelgänging","SSH Hijacking","Standard Non-Application Layer Protocol","Pass the Hash","Query Registry","Data Transfer Size Limits","Windows Remote Management","Path Interception","Endpoint Denial of Service","Uncommonly Used Port","BITS Jobs","Bypass User Account Control","Transmitted Data Manipulation","Data Encoding","Compile After Delivery","Compiled HTML File","Data from Information Repositories","Spearphishing via Service","Hardware Additions","Data Destruction","Install Root Certificate","Data Encrypted","Drive-by Compromise","Network Denial of Service","Hidden Files and Directories","Template Injection","Access Token Manipulation","Time Providers","Two-Factor Authentication Interception","Launch Agent","Create Account","Remote System Discovery","Network Service Scanning","Exfiltration Over Physical Medium","Remote File Copy","Windows Management Instrumentation Event Subscription","Launch Daemon","Group Policy Modification","XSL Script Processing","Dynamic Data Exchange","Fallback Channels","NTFS File Attributes","System Time Discovery","Service Execution","PowerShell","Inhibit System Recovery","Signed Script Proxy Execution","Custom Command and Control Protocol","Commonly Used Port","Exploitation for Defense Evasion","Trusted Developer Utilities","Windows Admin Shares"]
ALL_TTPS = ["TA0006","TA0002","TA0040","TA0003","TA0004","TA0008","TA0005","TA0010","TA0007","TA0009","TA0011","TA0001","T1066","T1047","T1156","T1113","T1067","T1037","T1033","T1003","T1129","T1492","T1044","T1171","T1014","T1501","T1123","T1133","T1109","T1099","T1069","T1114","T1163","T1025","T1116","T1093","T1178","T1013","T1192","T1489","T1206","T1063","T1080","T1167","T1165","T1137","T1089","T1487","T1214","T1119","T1115","T1103","T1007","T1040","T1135","T1120","T1082","T1071","T1053","T1162","T1176","T1106","T1058","T1202","T1024","T1091","T1005","T1140","T1195","T1190","T1219","T1079","T1036","T1055","T1205","T1218","T1038","T1050","T1010","T1032","T1062","T1182","T1029","T1004","T1009","T1076","T1131","T1181","T1483","T1185","T1021","T1207","T1107","T1145","T1112","T1491","T1155","T1217","T1183","T1085","T1031","T1092","T1222","T1179","T1019","T1042","T1117","T1054","T1108","T1193","T1215","T1101","T1177","T1125","T1144","T1045","T1016","T1198","T1087","T1090","T1059","T1482","T1175","T1020","T1070","T1083","T1138","T1191","T1188","T1074","T1049","T1064","T1051","T1497","T1102","T1104","T1480","T1204","T1196","T1057","T1141","T1041","T1060","T1023","T1026","T1122","T1015","T1212","T1210","T1142","T1199","T1098","T1170","T1048","T1097","T1110","T1001","T1039","T1078","T1073","T1068","T1208","T1027","T1201","T1187","T1486","T1488","T1174","T1002","T1081","T1128","T1056","T1203","T1168","T1100","T1186","T1184","T1095","T1075","T1012","T1030","T1028","T1034","T1499","T1065","T1197","T1088","T1493","T1132","T1500","T1223","T1213","T1194","T1200","T1485","T1130","T1022","T1189","T1498","T1158","T1221","T1134","T1209","T1111","T1159","T1136","T1018","T1046","T1052","T1105","T1084","T1160","T1484","T1220","T1173","T1008","T1096","T1124","T1035","T1086","T1490","T1216","T1094","T1043","T1211","T1127","T1077"]

#which techniques go with which tactics
TACTICS_TECHNIQUES_RELATIONSHIP_DF = pd.DataFrame({"TA0001":pd.Series(["T1133","T1192","T1091","T1195","T1190","T1193","T1199","T1078","T1194","T1200","T1189"]),
                                        "TA0002":pd.Series(["T1047","T1129","T1121","T1053","T1106","T1218","T1153","T1152","T1155","T1085","T1117","T1177","T1059","T1191","T1064","T1204","T1196","T1072","T1170","T1061","T1154","T1203","T1168","T1028","T1223","T1151","T1220","T1173","T1035","T1086","T1216","T1118","T1127"]),
                                        "TA0003":pd.Series(["T1156","T1067","T1037","T1161","T1150","T1044","T1501","T1133","T1109","T1163","T1013","T1180","T1165","T1137","T1103","T1053","T1162","T1176","T1058","T1205","T1038","T1050","T1062","T1182","T1004","T1131","T1152","T1183","T1031","T1179","T1019","T1042","T1164","T1108","T1215","T1101","T1177","T1198","T1138","T1060","T1023","T1122","T1015","T1098","T1157","T1078","T1154","T1128","T1168","T1166","T1100","T1034","T1197","T1158","T1209","T1159","T1136","T1084","T1160"]),
                                        "TA0004":pd.Series(["T1150","T1044","T1178","T1013","T1206","T1165","T1103","T1053","T1058","T1055","T1038","T1050","T1182","T1181","T1183","T1179","T1138","T1015","T1169","T1157","T1078","T1068","T1166","T1100","T1034","T1088","T1134","T1160"]),
                                        "TA0005":pd.Series(["T1066","T1143","T1150","T1148","T1006","T1014","T1109","T1099","T1116","T1093","T1121","T1089","T1202","T1140","T1036","T1055","T1205","T1218","T1038","T1009","T1181","T1152","T1207","T1107","T1112","T1183","T1085","T1222","T1117","T1054","T1108","T1144","T1045","T1198","T1070","T1191","T1064","T1497","T1102","T1480","T1196","T1122","T1149","T1170","T1078","T1073","T1027","T1186","T1197","T1088","T1147","T1500","T1223","T1146","T1130","T1158","T1221","T1134","T1151","T1126","T1484","T1220","T1096","T1216","T1118","T1211","T1127"]),
                                        "TA0006":pd.Series(["T1003","T1171","T1167","T1214","T1040","T1139","T1145","T1179","T1141","T1212","T1142","T1098","T1110","T1208","T1187","T1174","T1081","T1056","T1111"]),
                                        "TA0007":pd.Series(["T1033","T1069","T1063","T1007","T1040","T1135","T1120","T1082","T1010","T1217","T1016","T1087","T1482","T1083","T1049","T1497","T1057","T1201","T1012","T1018","T1046","T1124"]),
                                        "TA0008":pd.Series(["T1037","T1080","T1017","T1091","T1076","T1021","T1155","T1175","T1051","T1072","T1210","T1097","T1184","T1075","T1028","T1105","T1077"]),
                                        "TA0009":pd.Series(["T1113","T1123","T1114","T1025","T1119","T1115","T1005","T1185","T1125","T1074","T1039","T1056","T1213"]),
                                        "TA0010":pd.Series(["T1029","T1011","T1020","T1041","T1048","T1002","T1030","T1022","T1052"]),
                                        "TA0011":pd.Series(["T1172","T1071","T1024","T1219","T1079","T1205","T1032","T1483","T1092","T1090","T1188","T1102","T1104","T1026","T1001","T1095","T1065","T1132","T1105","T1008","T1094","T1043"]),
                                        "TA0040":pd.Series(["T1492","T1489","T1487","T1491","T1486","T1488","T1499","T1494","T1493","T1496","T1485","T1498","T1495","T1490"])
                                        })

In [ ]:
df2 = df
df2 = df2.drop("Text", 1)
columns = []
for column in df2:
    if df2.groupby(column).size()[1] > 4:
        print(df2.groupby(column).size())
    else:
        columns.append(column)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


TA0006
0    1184
1     306
dtype: int64
TA0002
0    1002
1     488
dtype: int64
TA0040
0    1415
1      75
dtype: int64
TA0003
0    900
1    590
dtype: int64
TA0004
0    1087
1     403
dtype: int64
TA0008
0    1147
1     343
dtype: int64
TA0005
0    722
1    768
dtype: int64
TA0010
0    1367
1     123
dtype: int64
TA0007
0    1134
1     356
dtype: int64
TA0009
0    1254
1     236
dtype: int64
TA0011
0    1084
1     406
dtype: int64
TA0001
0    1288
1     202
dtype: int64
T1066
0    1476
1      14
dtype: int64
T1047
0    1428
1      62
dtype: int64
T1113
0    1391
1      99
dtype: int64
T1067
0    1479
1      11
dtype: int64
T1037
0    1482
1       8
dtype: int64
T1033
0    1399
1      91
dtype: int64
T1003
0    1376
1     114
dtype: int64
T1129
0    1484
1       6
dtype: int64
T1492
0    1485
1       5
dtype: int64
T1044
0    1484
1       6
dtype: int64
T1171
0    1475
1      15
dtype: int64
T1014
0    1474
1      16
dtype: int64
T1501
0    1482
1       8
dtype: int64
T1123
0    1462
1

In [ ]:
for i in range(len(columns)):
    df = df.drop(columns[i], 1)
df = df.drop(df.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]], axis=1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Text,T1066,T1047,T1113,T1067,T1037,T1033,T1003,T1129,T1492,...,T1096,T1124,T1035,T1086,T1490,T1094,T1043,T1211,T1127,T1077
0,Talos Blog || Cisco Talos Intelligence Group -...,1,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,OilRig Actors Provide a Glimpse into Developme...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hogfish Redleaves Malware Threat Analysis I Ac...,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Scheduled Transfer - Enterprise | MITRE ATT&CK...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Winlogon Helper DLL - Enterprise | MITRE ATT&C...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from bs4 import BeautifulSoup             
import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english") )

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#raw_report, a single text document 
#bad_words, a list of words to be removed from the texts
# also removes stop words, html, and non-letters
def reports_to_words(raw_report, bad_words):

    # 1. Remove HTML
    review_text = BeautifulSoup(raw_report).get_text() 
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    word = letters_only.lower()


    word = word.split()

    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english")) 
    # add words picked to stop word set that have a low mutual information score or a low frequency
    stops.update(bad_words)

    # 5. Remove stop words
    meaningful_words = [w for w in word if not w in stops] 
  
    #
    # 6. Create WordNetLemmatizer object
    wnl = WordNetLemmatizer()
    
    lemmatized_string = [wnl.lemmatize(words) for words in meaningful_words]
    #
    # 7. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( lemmatized_string ))

In [ ]:
#raw_report, a single text document 
# removes stop words, html, and non-letters
def init_to_words(raw_report):

    #raw_report = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'repurl', raw_report)
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_report).get_text() 
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    word = letters_only.lower()

    #for i in range(len(bad_words)):
     # if bad_words[i] in word:
      #  print(bad_words[i])

    word = word.split()

    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english")) 

    # 5. Remove stop words
    meaningful_words = [w for w in word if not w in stops] 
  
    #
    # 6. Create WordNetLemmatizer object
    wnl = WordNetLemmatizer()
    
    lemmatized_string = [wnl.lemmatize(words) for words in meaningful_words]
    #
    # 7. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( lemmatized_string ))

In [ ]:
# Get the number of reports based on the dataframe column size
num_reports = len(df)

# Initialize an empty list to hold the clean reports
clean_train_reports = []

# Loop over each report; create an index i that goes from 0 to the length
# of the report list
for i in range(0, num_reports):
    # Call our function for each one, and add the result to the list of
    # clean reports
    clean_train_reports.append(init_to_words(str(df['Text'][i])))

In [ ]:
#unlabelled malware reports for snorkel
file = open("/content/gdrive/MyDrive/thesis_work/unlabelled_reports.txt","r")
contents = []
reports = []
for line in file:
    contents.append(line)
    if line == '+++++++++\n':
        text = ''
        for x in range(len(contents)):
            text = text + contents[x]
        reports.append(text)
        contents = []    
file.close()

In [ ]:
unlabelled = pd.DataFrame(reports, columns=['text'])
unlabelled.head()

,text
0,"By Gabor Szappanos, Principal ResearcherPlugX ..."
1,\nThe 'Madi' infostealers - a detailed analysi...
2,\nBy GReATRecent Cloud Atlas activitysecurelis...
3,\nSNAKE CAMPAIGN& CYBER ESPIONAGE TOOLKIT 2EXE...
4,\nREPORTSUPPLY CHAIN ANALYSIS:From Quartermast...


In [ ]:
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier
import re

#define label mappings for snorkel for convenience 
FALSE = 0
TRUE = 1
ABSTAIN = -1

def keyword_lookup(x, keywords, label):
    match = re.search(fr"{keywords}", x.text.lower())
    #if match is not None:
    if (keywords in x.text.lower()):
        #print(keywords)
        #print(match.group(0))
        return label
    else:
        return ABSTAIN


def make_keyword_lf(keywords, label=TRUE):
    return LabelingFunction(
        name=f"keyword_{keywords}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label),
    )


In [ ]:
def snorkel(df, col):
  #get reports with the behavior
      bowa = df.loc[df[col] == 1]
      bowa.reset_index(drop=True, inplace=True)
      bowa.head()
      #get reports without behavior
      bowna = df.loc[df[col] == 0]
      bowna.reset_index(drop=True, inplace=True)
      bowna.head()

      #clean reports with behavior
      # Get the number of reports based on the dataframe column size
      num_reports = len(bowa)

      # Initialize an empty list to hold the clean reports
      clean_train_reports = []

      # Loop over each report; create an index i that goes from 0 to the length
      # of the exploit list
      for i in range(0, num_reports):
          # Call our function for each one, and add the result to the list of
          # clean exploits
          clean_train_reports.append(init_to_words(str(bowa['Text'][i])))

      #clean reports without behavior

      num_reports = len(bowna)

      clean_train_reports2 = []

      for i in range(0, num_reports):
          clean_train_reports2.append(init_to_words(str(bowna['Text'][i])))

      #create BOW for reports with behavior
      vectorizer = CountVectorizer(analyzer = "word",   \
                                tokenizer = None,    \
                                preprocessor = None,
                                max_features = 2000) 

      train_data_features = vectorizer.fit_transform(clean_train_reports)


      # Numpy arrays are easy to work with, so convert the result to an 
      # array

      train_data_features = train_data_features.toarray()

      #create BOW for reports without behavior

      train_data_features2 = vectorizer.transform(clean_train_reports2)

      train_data_features2 = train_data_features2.toarray()

      #create vectorizer for reports without behavior

      vectorizer2 = CountVectorizer(analyzer = "word",   \
                                tokenizer = None,    \
                                preprocessor = None,
                                max_features = 2000) 

      train_data_features5 = vectorizer2.fit_transform(clean_train_reports2)

      train_data_features5 = train_data_features5.toarray()

      #create BOW from second vectorizer for reports with behavior

      train_data_features6 = vectorizer2.transform(clean_train_reports)

      train_data_features6 = train_data_features6.toarray()
      #get vocab from first vectorizer
      vocab = vectorizer.get_feature_names()
      #get counts for each word in vocab for reports with behavior
      dist = np.sum(train_data_features, axis=0)

      vocab2 = vectorizer.get_feature_names()
      #get counts for each word in vocab for reports without behavior
      dist2 = np.sum(train_data_features2, axis=0)

      #get vocab from second vectorizer
      vocab3 = vectorizer2.get_feature_names()
      #get counts for each word in vocab3 for reports without behavior
      dist3 = np.sum(train_data_features5, axis=0)

      vocab4 = vectorizer2.get_feature_names()
      #get counts for each word in vocab3 for reports with behavior
      dist4 = np.sum(train_data_features6, axis=0)
      #ratio of words between reports with behavior and without
      freq = dist / dist2
      #ratio of words between reports without behavior and with
      freq2 = dist3 / dist4
      high_fr2_nb = []
      high_fr_nb = []
      high_fr = []
      high_fr2 = []
      
      if(len(bowa) >= 30):
            #make a list of high freq words in reports without behavior
            
            for i in range(len(freq2)):
                if(freq2[i] == np.inf):
                    freq2[i] = 1
                    if (dist3[i] >= 400):
                        #if(vocab3[i] != 'hich'): 
                        high_fr2_nb.append(' ' + vocab3[i] + ' ')
           
            for i in range(len(freq2)):
                if((freq2[i] >= 250) and ((dist3[i] / freq2[i]) <= 1)):
                    #if(vocab3[i] != 'nging'):
                    high_fr_nb.append(' ' + vocab3[i] + ' ')
        
            #make list of high freq words in reports with behavior
            high_fr2 = []
            for i in range(len(freq)):
                if(freq[i] == np.inf):
                    freq[i] = 1
                    if (dist[i] >= 20):
                        #if(vocab[i] != 'resolv'): 
                        high_fr2.append(' ' + vocab[i] + ' ')

            high_fr = []
            for i in range(len(freq)):
                if((freq[i] >= 20) and ((dist[i] / freq[i]) <= 3)):
                    #if(vocab[i] != 'xda'):
                    high_fr.append(' ' + vocab[i]+ ' ')
      elif(len(bowa) >= 20):
            #make a list of high freq words in reports without behavior
            
            for i in range(len(freq2)):
                if(freq2[i] == np.inf):
                    freq2[i] = 1
                    if (dist3[i] >= 600):
                        #if(vocab3[i] != 'hich'): 
                        high_fr2_nb.append(' ' + vocab3[i] + ' ')
           
            for i in range(len(freq2)):
                if((freq2[i] >= 500) and ((dist3[i] / freq2[i]) <= 1)):
                    #if(vocab3[i] != 'nging'):
                    high_fr_nb.append(' ' + vocab3[i] + ' ')
        
            #make list of high freq words in reports with behavior
            high_fr2 = []
            for i in range(len(freq)):
                if(freq[i] == np.inf):
                    freq[i] = 1
                    if (dist[i] >= 10):
                        #if(vocab[i] != 'resolv'): 
                        high_fr2.append(' ' + vocab[i] + ' ')

            high_fr = []
            for i in range(len(freq)):
                if((freq[i] >= 10) and ((dist[i] / freq[i]) <= 3)):
                    #if(vocab[i] != 'xda'):
                    high_fr.append(' ' + vocab[i]+ ' ')
      elif(len(bowa) >= 10):
            #make a list of high freq words in reports without behavior
            
            for i in range(len(freq2)):
                if(freq2[i] == np.inf):
                    freq2[i] = 1
                    if (dist3[i] >= 800):
                        #if(vocab3[i] != 'hich'): 
                        high_fr2_nb.append(' ' + vocab3[i] + ' ')
           
            for i in range(len(freq2)):
                if((freq2[i] >= 700) and ((dist3[i] / freq2[i]) <= 1)):
                    #if(vocab3[i] != 'nging'):
                    high_fr_nb.append(' ' + vocab3[i] + ' ')
        
            #make list of high freq words in reports with behavior
            high_fr2 = []
            for i in range(len(freq)):
                if(freq[i] == np.inf):
                    freq[i] = 1
                    if (dist[i] >= 5):
                        #if(vocab[i] != 'resolv'): 
                        high_fr2.append(' ' + vocab[i] + ' ')

            high_fr = []
            for i in range(len(freq)):
                if((freq[i] >= 5) and ((dist[i] / freq[i]) <= 3)):
                    #if(vocab[i] != 'xda'):
                    high_fr.append(' ' + vocab[i]+ ' ')
          
      #make snorkel keywords from high freq words
      lfs = []
      print(col)
      print('pos words')
      print(high_fr)
      print(high_fr2)
      print('neg words')
      print(high_fr_nb)
      print(high_fr2_nb)
      for i in range(len(high_fr)):
          keyword = make_keyword_lf(high_fr[i])
          lfs.append(keyword)
          #weights.append(.7)

      for i in range(len(high_fr2)):
          keyword = make_keyword_lf(high_fr2[i])
          lfs.append(keyword)
          #weights.append(.7)
      if(len(bowa) >= 10):
          for i in range(len(high_fr_nb)):
              keyword = make_keyword_lf(high_fr_nb[i], FALSE)
              lfs.append(keyword)
              #weights.append(.2)

          for i in range(len(high_fr2_nb)):
              keyword = make_keyword_lf(high_fr2_nb[i], FALSE)
              lfs.append(keyword)
              #weights.append(.2)
      return lfs

In [ ]:
from TextFeatureSelection import TextFeatureSelection
from sklearn.model_selection import KFold
from pandas.io.parsers.python_parser import count_empty_vals
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import math
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

total_pred = []
total_act = []
count = 0
#loop through each tactic
for col in df:  
  if col != 'Text':
    recall = []
    prec = []
   
    #get train and test set
    X_train, X_test, y_train, y_test = train_test_split(clean_train_reports, df[col], test_size=0.20, random_state=42, stratify = df[col])

    y_train.reset_index(drop=True, inplace=True)
    pos = pd.DataFrame (X_train, columns = ['Text'])
    pos = pos.loc[y_train == 1]
    pos_texts = pos['Text'].values.tolist()

    #add all the words with a low frequency for that tactic to the list of words to remove
    #bad_words = bad_words + all_pos_stops[count] + all_pos_stops2[count] + all_pos_stops3[count]

    #obtain labeling functions from snorkel
    if(len(pos_texts) >= 10):
      lfs = snorkel(df, col)


    # Get the number of unlabelled reports based on the dataframe column size
    num_reports = len(unlabelled)

    # Initialize an empty list to hold the clean exploits
    unlabelled_reports = []

    # Loop over each unlabelled report; create an index i that goes from 0 to the length
    # of the unlabelled list
    for i in range(0, num_reports):
        # Call our function for each one, and add the result to the list of
        # clean reports
        unlabelled_reports.append(init_to_words(str(unlabelled['text'][i])))

    unlabelled2 = pd.DataFrame(unlabelled_reports, columns = ['text'])
    if(len(pos_texts) >= 10):
      # Apply the LFs to the unlabeled training data
      applier = PandasLFApplier(lfs)
      L_train = applier.apply(unlabelled2)

      # Train the label model and compute the training labels
      label_model = LabelModel(cardinality=2, verbose=True)
      label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
      unlabelled2["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

      unlabelled2['label'] = unlabelled2['label'].replace(-1, 0)
      #df_train2 = df_train2[df_train2.label != -1]
      unlabelled2 = unlabelled2.reset_index(drop=True)

    # Get the number of reports based on the dataframe column size
    num_reports = len(df)

    # Initialize an empty list to hold the clean exploits
    clean_train_reports2 = []

    #remove all stop words and words with low mutual information score from the texts
    # Loop over each report; create an index i that goes from 0 to the length
    # of the report list
    for i in range(0, num_reports):
        # Call our function for each one, and add the result to the list of
        # clean report
        clean_train_reports2.append(init_to_words(str(df['Text'][i])))

    #rebreak the cleaned text into the training and test sets
    X_train, X_test, y_train, y_test = train_test_split(clean_train_reports2, df[col], test_size=0.20, random_state=42, stratify = df[col])


    #create a tfidf vector representation
    vectorizer = TfidfVectorizer(analyzer = "word",   \
                                tokenizer = None,    \
                                preprocessor = None) 
    


    #fit the tfidf representation to the training set
    train_data_features = vectorizer.fit_transform(X_train)
    train_data_features = train_data_features.toarray()

    #create tfidf representation for the test set
    test_data_features = vectorizer.transform(X_test)
    test_data_features = test_data_features.toarray()

    if(len(pos_texts) >= 10):

      #create tfidf for the unlabelled data
      unlabelled_features = vectorizer.transform(unlabelled_reports)
      unlabelled_features = unlabelled_features.toarray()

      train_data_features = train_data_features.transpose()
      unlabelled_features = unlabelled_features.transpose()
      #combine snorkel data with training set
      combined_features = np.concatenate((train_data_features, unlabelled_features), axis=1)
      combined_features = combined_features.transpose()

      #concatenate labels
      y_train = y_train.tolist()
      print(y_train)
      y_train2 = unlabelled2['label'].tolist()
      joined_labels = y_train + y_train2

      clf = LinearSVC()

      #fit linear svc to training set
      clf.fit(combined_features, joined_labels)
      #make predictions
      pred = clf.predict(test_data_features)
    else:
      clf = LinearSVC()

      #fit linear svc to training set
      clf.fit(train_data_features, y_train)
      #make predictions
      pred = clf.predict(test_data_features)

    print('prec: ', precision_score(y_test, pred))
    print('recall: ', recall_score(y_test, pred))
    #append predictions for each technique
    total_pred.append(pred)
    total_act.append(y_test)



    print(count)
    count += 1

T1066
pos words
[' attacksaga ', ' cfireeye ', ' cmessageprocessingsystem ', ' cstring ', ' gifas ', ' keystore ', ' keytool ', ' prepend ', ' screensaverautorun ', ' shellautorun ', ' startupautorun ', ' taskhost ', ' vaeit ', ' vine ', ' withyourface ']
[' adeel ', ' ameteksen ', ' autorunmanager ', ' capstone ', ' cdoon ', ' cexecutionsubsystem ', ' ckorban ', ' clng ', ' cniihm ', ' dllinjector ', ' enumusers ', ' fsstorage ', ' htmlsource ', ' hurix ', ' injectmanager ', ' lint ', ' ltmanager ', ' ltmessageprocessing ', ' ltnamedpipe ', ' movabs ', ' msoftupdates ', ' mtrv ', ' neteease ', ' regstorage ', ' remoteimport ', ' resultqueue ', ' rootstoragepath ', ' rportfwd ', ' sakurel ', ' solidloop ', ' spawnto ', ' stageless ', ' taskqueue ', ' tnptransport ', ' topsec ', ' ultimatecomsup ', ' xrat ', ' xsid ']
neg words
[' bank ', ' emotet ', ' facebook ', ' fin ', ' forest ', ' kaspersky ', ' mandiant ', ' node ', ' offset ', ' packet ', ' pm ', ' push ', ' south ', ' xaa ', ' 

100%|██████████| 503/503 [00:05<00:00, 92.18it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:01<00:00, 427.98epoch/s]
INFO:root:Finished Training


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 503/503 [00:03<00:00, 127.46it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
  7%|▋         | 34/500 [00:00<00:01, 338.58epoch/s]INFO:root:[50 epochs]: TRAIN:[loss=0.010]
INFO:root:[100 epochs]: TRAIN:[loss=0.008]
 36%|███▋      | 182/500 [00:00<00:00, 653.53epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.006]
INFO:root:[250 epochs]: TRAIN:[loss=0.005]
 66%|██████▌   | 329/500 [00:00<00:00, 702.93epoch/s]INFO:root:[350 epochs]: TRAIN:[loss=0.005]
INFO:root:[400 epochs]: TRAIN:[loss=0.005]
100%|██████████| 500/500 [00:00<00:00, 691.78epoch/s]
INFO:root:Finished Training


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 503/503 [00:09<00:00, 50.38it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:02<00:00, 205.26epoch/s]
INFO:root:Finished Training


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 503/503 [00:04<00:00, 100.63it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
 28%|██▊       | 139/500 [00:00<00:00, 492.83epoch/s]INFO:root:[150 epochs]: TRAIN:[loss=0.016]
INFO:root:[200 epochs]: TRAIN:[loss=0.016]
100%|██████████| 500/500 [00:00<00:00, 529.29epoch/s]
INFO:root:Finished Training


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
total_pred = np.array(total_pred)
total_act = np.array(total_act)

In [ ]:
#techniques snorkel
from sklearn.metrics import classification_report 
report = classification_report(total_act, total_pred, output_dict=True)   
df2 = pd.DataFrame(report).transpose()
df2.tail()

,precision,recall,f1-score,support
297,1.000000,0.134021,0.236364,97.0
micro avg,0.573991,0.104575,0.176918,1224.0
macro avg,0.151222,0.048560,0.062515,1224.0
weighted avg,0.765404,0.104575,0.172170,1224.0
samples avg,0.277845,0.102308,0.139381,1224.0
